### Generate datapoints

In [ ]:
import numpy as np

silhouette = np.array([
    [0, 0, 0, 1, 1, 1, 1, 0, 0, 0],
    [0, 0, 1, 1, 1, 1, 1, 1, 0, 0],
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
    [1, 1, 1, 0, 0, 0, 1, 1, 1, 1],
    [1, 1, 0, 0, 0, 0, 0, 1, 1, 1],
    [1, 1, 1, 0, 0, 0, 1, 1, 1, 1],
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
    [0, 0, 1, 1, 1, 1, 1, 1, 0, 0],
    [0, 0, 0, 1, 1, 1, 1, 0, 0, 0],
    [0, 0, 0, 0, 1, 1, 0, 0, 0, 0]
])

# 5 silhouettes per datapoint
# Same for all, for the sake of simplicity
gait_sequence = np.array([silhouette for _ in range(5)])

datapoints = []
for i in range(10):
    # Subject ID assigned to each
    subject_id = f'00{i%3 + 1}'
    # Same for all, for the sake of simplicity
    conditions = 'concrete,backpack,sneakers,1_week'
    # Create the datapoint as a structured array
    datapoint = np.array([(subject_id, conditions, gait_sequence)], 
                         dtype=[('subject_id', 'U10'), ('conditions', 'U50'), ('gait_sequence', 'O')])
    # Append the datapoint to our list
    datapoints.append(datapoint)

dataset = np.concatenate(datapoints)

# print(dataset)

### Convert to vectors

Here, a matrix is formed where each row vector is a datapoint.

In [ ]:
flattened_sequences = [datapoint['gait_sequence'].flatten() for datapoint in dataset]
matrix = np.vstack(flattened_sequences)

# np.set_printoptions(threshold=np.inf)
# print(matrix)